In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [25]:
##Loading All trained Models
from tensorflow.keras.models import load_model
model = load_model('model.h5')

#load encoders and Scalers
with open('label_encoder_gender.pkl', 'rb') as f:
    label_encoder_gender = pickle.load(f)
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)
with open('ohe_geography.pkl', 'rb') as f:
    label_encoder_geography = pickle.load(f)



In [26]:
#Example input data
data = {    'CreditScore': 600,
    'Geography': 'France',
    "Gender" : 'Male',
    'Age': 40,
    'Tenure': 5,
    'Balance': 10000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}
print(label_encoder_gender.classes_)  # Should show ['Female', 'Male'] or similar


['Female' 'Male']


In [27]:
##Encode the input data
geography_encoded = label_encoder_geography.transform([[data['Geography']]]).toarray()
#To DataFrame
geography_df = pd.DataFrame(geography_encoded, columns=label_encoder_geography.get_feature_names_out(['Geography']))
geography_df

/Users/mariebiskoot/Coding/Ann_Imp/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [28]:
#Combine all data
data = pd.concat([pd.DataFrame(data, index=[0]), geography_df], axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,Male,40,5,10000,2,1,1,50000,1.0,0.0,0.0


In [29]:
data.drop(['Geography'], axis=1, inplace=True)


In [30]:
##Encode Categorical data
data['Gender'] = label_encoder_gender.transform(data['Gender'])
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,5,10000,2,1,1,50000,1.0,0.0,0.0


In [31]:
#scale the data
data = scaler.transform(data)
data

array([[-0.53598516,  0.91324755,  0.10479359, -0.00134472, -1.05836066,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [32]:
##Predict
prediction = model.predict(data)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
[[0.02916195]]


In [33]:
if prediction[0][0] > 0.5:
    print("Customer is likely to leave the bank.")
else:
    print("Customer is likely to stay with the bank.")

Customer is likely to stay with the bank.
